In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
%matplotlib inline
from matplotlib import pyplot as plt

torch.set_printoptions(edgeitems=2, linewidth=75)
data_path = '../data-unversioned/p1ch7/'

In [ ]:
cifar10 = datasets.CIFAR10(data_path, train=True, download= True)           # Training dataset....
cifar10_val = datasets.CIFAR10(data_path, train=False, download= True)      # Dataset for validation....

In [3]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'] # class definations...

In [4]:
# Splitting dataset into training and validation set...

# Trainig data...
cifar10_Train = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Validation data...
transformed_cifar10_Val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


In [5]:
# Hyperparameters...
learning_Rate = 1e-2
num_epochs = 300
loss_fnc = nn.CrossEntropyLoss()

# Defining the model...
model21 = nn.Sequential(
                       nn.Linear(3072, 512),
                       nn.Tanh(),
                       nn.Linear(512,10))

# Optimizer function...
optimizer = optim.SGD(model21.parameters(), lr = learning_Rate)

#Training Observations:

- It took about 56m.58s to train with 300 epochs on colab, with GPU accelration.
- It took about 2:30 hrs to train with 300 epochs on colab, without GPU accelration.
- It took about 35m to train with 200 epochs on colab, with GPU accelration.
- Training loss at 200 epochs is less than 0.012718.
- Training loss at 300 epochs is less then 0.006778.

In [6]:
# Dividing dataset into mini-batches...
train_mbload = torch.utils.data.DataLoader(cifar10_Train, batch_size = 64, shuffle=True)

# Training loop...
for epoch in range(num_epochs):
    for imgs, labels in train_mbload:
        outputs = model21(imgs.view(imgs.shape[0], -1))
        loss = loss_fnc(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch == 1 or epoch % 30 == 0:
       print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.815016
Epoch: 1, Loss: 1.910540
Epoch: 30, Loss: 0.910494
Epoch: 60, Loss: 0.169447
Epoch: 90, Loss: 0.042646
Epoch: 120, Loss: 0.027814
Epoch: 150, Loss: 0.016693
Epoch: 180, Loss: 0.012718
Epoch: 210, Loss: 0.010674
Epoch: 240, Loss: 0.007415
Epoch: 270, Loss: 0.006778


## Accuracy and Analysis:

- As, mentioned below Trianing Accuracy: 1.000, and Validation Accuracy: 0.462200.
- Its apparent from accuracy values that developed model21 is overfitting. As it has perfect accuracy for training set and bad accuracy on validation set.
- One reason for bad accuracy on validation set is that, model might have memorized and fix the paramter values learned on training set.
- Also, absense of transiation invariance in fully connected model might have effected accuracy on evaluation set.  


In [7]:
# Checking the accuracy of model...
train_accuload = torch.utils.data.DataLoader(cifar10_Train, batch_size = 64, shuffle=False)
Valid_accuload = torch.utils.data.DataLoader(transformed_cifar10_Val, batch_size = 64, shuffle=False)

# Initializing the variables...
training_correct = 0
validation_correct = 0
training_total = 0
validation_total = 0

# Checking on the training dataset...
with torch.no_grad():
  for imgs, labels in train_accuload:
    train_results = model21(imgs.view(imgs.shape[0],-1))
    _, train_classified = torch.max(train_results, dim =1)
    training_total += labels.shape[0]
    training_correct += int((train_classified == labels).sum())

print('Training Accuracy: %f' % (training_correct/training_total))

# Checking on the validation dataset...
with torch.no_grad():
  for imgs, labels in Valid_accuload:
    Valid_results = model21(imgs.view(imgs.shape[0],-1))
    _, Valid_classified = torch.max(Valid_results, dim =1)
    validation_total += labels.shape[0]
    validation_correct += int((Valid_classified == labels).sum())

print('Validation Accuracy: %f' % (validation_correct/validation_total))

Training Accuracy: 1.000000
Validation Accuracy: 0.462200
